# Bulk RNA Seq Analysis
If you are working with human or mouse data, then you should only need to edit the "Create config.txt and samples.txt" and "Download Fastq files" code blocks. If you are working with the data from another species, then you will have to edit the "download gencode files" block accordingly. 

# Create config.txt and samples.txt
This code block creates a config file and a list of samples so that most of the rest of the script can be autonomous. 

Instructions:
- Edit species to reflect human or mouse.
- Edit version to reflect the current gencode release. Do NOT include the 'M' prefix to the mouse release. Note: you can check the current releases here https://www.gencodegenes.org/. 
- Edit the fastqExtension. If your fastq file names have a ton of junk in the middle of them, this refers to that junk. Example below.
- Edit the list of your samples. Example below.

Example:
Let's say you have three paired end samples to quantify: BL, H6, and L24. Your list of fastqs looks like this:
- BLa739_9747_AHFD7D_1.fastq
- BLa739_9747_AHFD7D_2.fastq
- H6a739_9747_AHFD7D_1.fastq
- H6a739_9747_AHFD7D_2.fastq
- L24a739_9747_AHFD7D_1.fastq
- L24a739_9747_AHFD7D_2.fastq

Your variables in the script below would then be:
fastqExtension = 'a739_9747_AHDFD7D'
samples = {'BL', 'H6', 'L6']

Note: If you have paired end reads, do NOT list each fastq separately! List each SAMPLE separately.

In [38]:
species = 'human'
gencodeVersion = 44
fastqExtension = 'WD0719'
samples = ['S1_R','S2_R','S3_R','S4_R','S5_R','S6_R','S7_R','S8_R','S9_R','S10_R','S11_R','S12_R','S13_R','S14_R']

#don't edit this next part
if species == 'human':
    speciesfull = 'human'
    specieslow = 'h'
    speciesup = ''
    release = 38
elif species == 'mouse':
    speciesfull = 'mouse'
    specieslow = 'm'
    speciesup = 'M'
    release = 39
else:
    print('Error: you formatted your species entry incorrectly')

config = open("config.txt", "w")
configelements = [fastqExtension, gencodeVersion, speciesfull, specieslow, speciesup, release]
for line in configelements:
    config.write("%s\n" % line)
config.close()

samps = open("samples.txt", "w")
for line in samples:
    samps.write("%s\n" % line)
samps.close()
print('Done')

Done


# Download Fastq files
This is something you'll need to more or less format yourself. Bash in jupyter gets grumpy about in-line comments, so here's what the code does:
- Sets up two variables, a LIST of your samples, and the fastq extension, FQEXT that you set up in the config files. 
- Sets up a loop that will cycle through your samples.

Here's what you need to do:
- Replace "#download fastq command" with the command to download your fastqs. There is an echo command with some sample formatting included.
- If you have paired end reads, make sure you include two separate download commands in your loop to account for two downloads per sample.
- Make sure you download them in zipped format and leave them zipped for later steps, as well as for minimizing storage space.

In [37]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
LIST=$(cat samples.txt)
FQEXT=$(sed -n '1p' config.txt)
mkdir fastq
cd fastq
for i in ${LIST}
    do
        #download fastq command
        #wget --user=P2006072_08012023 --password='14Hw!mnZ^G1j' https://portal-us.medgenome.com/P2006072_08012023/FASTQ/${FQEXT}${i}.fastq.gz
        echo 'fastq/'${i}${FQEXT}'.fq.gz downloaded'
    done

fastq/controlA_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded
fastq/controlB_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded
fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded
fastq/sixB_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded
fastq/tfourA_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded
fastq/tfourB_CKDL230019755-1A_H75TNDSX7_L2_2.fq.gz downloaded


# Download gencode files, generate decoys.txt + gene_map.csv, and run salmon index
If you did things right and I did things right, you should be able to just run this code block if you're working with mouse or human data. Good luck!

That being said, you MUST verify that each step of this script actually worked before the subsequent steps will run correctly. If you just hit "run" and hope for the best, you may not catch your error until way downstream when all your files are empty. 

If you're NOT working with mouse or human data, you need to edit the wget lines to whatever will download your transcript and primary assembly files, as well as any subsequent file paths that point to the downloaded files.

In [39]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate salmon
VERS=$(sed -n '2p' config.txt)
SPECIESFULL=$(sed -n '3p' config.txt)
SPECIESlow=$(sed -n '4p' config.txt)
SPECIESup=$(sed -n '5p' config.txt)
RELEASE=$(sed -n '6p' config.txt)

#download needed gencode files
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_${SPECIESFULL}/release_${SPECIESup}${VERS}/gencode.v${SPECIESup}${VERS}.transcripts.fa.gz
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_${SPECIESFULL}/release_${SPECIESup}${VERS}/GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz

#generate decoys.txt from the transcript and primary assembly files
grep "^>" <(gunzip -c GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz) | cut -d " " -f 1 > decoys.txt
sed -i.bak -e 's/>//g' decoys.txt
cat gencode.v${SPECIESup}${VERS}.transcripts.fa.gz GRC${SPECIESlow}${RELEASE}.primary_assembly.genome.fa.gz > gentrome.fa.gz

#create gene_map.csv
gunzip gencode.v${SPECIESup}${VERS}.transcripts.fa.gz
echo ${SPECIESlow}
if [[ ${SPECIESlow} == "h" ]]; then
    echo 'human'
    grep -P -o 'ENST\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > enst.txt
    grep -P -o 'ENSG\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > ensg.txt 
else
    echo 'mouse'
    grep -P -o 'ENSMUST\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > enst.txt
    grep -P -o 'ENSMUSG\d{11}' gencode.v${SPECIESup}${VERS}.transcripts.fa > ensg.txt
fi
paste -d ',' enst.txt ensg.txt > gene_map.csv
gzip gencode.v${SPECIESup}${VERS}.transcripts.fa

#run salmon indexer
salmon index -t gentrome.fa.gz -d decoys.txt -p 12 -i salmon_index --gencode
echo 'Done'

--2023-08-14 13:29:35--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.transcripts.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82601810 (79M) [application/x-gzip]
Saving to: ‘gencode.v44.transcripts.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  274K 4m54s
    50K .......... .......... .......... .......... ..........  0%  547K 3m41s
   100K .......... .......... .......... .......... ..........  0%  549K 3m16s
   150K .......... .......... .......... .......... ..........  0% 71.9M 2m27s
   200K .......... .......... .......... .......... ..........  0%  548K 2m27s
   250K .......... .......... .......... .......... ..........  0%  549K 2m27s
   300K .......... .......... .......... .......... ..........  0%  548K 2m27s
   350K .......... .......... .......... .......... 

h
human
Done


# Run Salmon quant on paired end reads
Run this code block if you have paired end reads.

Validate it step by step FIRST so that you don't end up with a dozen unzipped fastqs.

You may have to reformat the way the fastq's are named based on how your fastq names are formatted. Use echo statements to debug.

In [36]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate salmon
LIST=$(cat samples.txt)
FQEXT=$(sed -n '1p' config.txt)
for i in ${LIST}
    do
        gunzip fastq/${i}${FQEXT}1.fq.gz
        gunzip fastq/${i}${FQEXT}2.fq.gz
        salmon quant -i salmon_index -l A -1 fastq/${i}${FQEXT}1.fq -2 fastq/${i}${FQEXT}2.fq --validateMappings -o ${i}
        gzip fastq/${i}${FQEXT}1.fq
        gzip fastq/${i}${FQEXT}2.fq
    done

gzip: fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2_1.fq.gz: No such file or directory
gzip: fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2_2.fq.gz: No such file or directory
Version Info: This is the most recent version of salmon.
### salmon (selective-alignment-based) v1.10.1
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { salmon_index }
### [ libType ] => { A }
### [ mates1 ] => { fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2_1.fq }
### [ mates2 ] => { fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2_2.fq }
### [ validateMappings ] => { }
### [ output ] => { sixA }
Logs will be written to sixA/logs
[2023-07-14 12:49:45.781] [jointLog] [info] setting maxHashResizeThreads to 32
[2023-07-14 12:49:45.781] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2023-07-14 12:49:45.781] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly specified, it is being set to 0.65
[2023-07-1

CalledProcessError: Command 'b"source /bobross/miniconda3/etc/profile.d/conda.sh\nconda activate salmon\nLIST=$(cat samples.txt)\nFQEXT=$(sed -n '1p' config.txt)\nfor i in ${LIST}\n    do\n        gunzip fastq/${i}${FQEXT}_1.fq.gz\n        gunzip fastq/${i}${FQEXT}_2.fq.gz\n        salmon quant -i salmon_index -l A -1 fastq/${i}${FQEXT}_1.fq -2 fastq/${i}${FQEXT}_2.fq --validateMappings -o ${i}\n        gzip fastq/${i}${FQEXT}_1.fq\n        gzip fastq/${i}${FQEXT}_2.fq\n    done\n"' returned non-zero exit status 1.

# Run salmon quant on single end reads
Run this code block if you have single end reads.

Validate it step by step FIRST so that you don't end up with a dozen unzipped fastqs.

You may have to reformat the way the fastq's are named based on how your fastq names are formatted. Use echo statements to debug.

In [38]:
%%bash
source /bobross/miniconda3/etc/profile.d/conda.sh
conda activate salmon
LIST=$(cat samples.txt)
FQEXT=$(sed -n '1p' config.txt)
for i in ${LIST}
    do
        gunzip fastq/${i}${FQEXT}.fq.gz
        salmon quant -i salmon_index -l A -r fastq/${i}${FQEXT}.fq --validateMappings -o ${i}
        gzip fastq/${i}${FQEXT}.fq
    done

fastq/sixA_CKDL230019755-1A_H75TNDSX7_L2_2.fq
Process is interrupted.


# Generate SRARunTable.txt files
DeSeq2 will eventually rearrange your data so that it is alphabetical and then treats whatever comes first as the control. Make sure your controls are listed FIRST and are the first of your samples alphabetically. You may need to edit the names of your salmon outputs to reflect this. Edit these lists in the code block below according to the following:

- samples: This should be a list of your samples. Unless you need to make changes for alphabetization, this should match your sample list from the first code block.
- treatments: Every value is either "control" or "treatment" in this list. List positions correspond to sample list positions.
- conditions: Each value is an experiment-specific treatment label. These are used to group samples by condition. 
- cell_line: Each value is the cell line that the experiment sample was carried out in. 
- files: This should be your ONLY list of a different length that does NOT correspond to positions in the other lists. This should just be a list of conditions that exist in your experiment, excluding "control." However you format this should match what you tell R in subsequent analysis.

For an example, see the start to the code block below. 

In [29]:
samples = ['controlA','controlB','NonionA','NonionB','PBSA','PBSB','SM102eLNPA','SM102eLNPB','SM102RNAA','SM102RNAB','TCLeLNPA','TCLeLNPB','TCLRNAA','TCLRNAB']
treatments = ["control", "control", "treatment", "treatment", "treatment", "treatment", "treatment", "treatment","treatment","treatment","treatment","treatment","treatment","treatment"]
files = ["nonion","PBS","SM102eLNP","SM102RNA","TCLeLNP","TCLRNA"]
conditions = ["control", "control","nonion","nonion","PBS","PBS","SM102eLNP","SM102eLNP","SM102RNA","SM102RNA","TCLeLNP","TCLeLNP","TCLRNA","TCLRNA"]
cell_line = ['mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse','mouse']


SRT = open("SraRunTableall.txt", "w")
SRT.write("%s\n" % "sample name, treatment, cell_line")
for i in range(len(samples)):
    SRT.write(samples[i])
    SRT.write(', ')
    SRT.write(treatments[i])
    SRT.write(', ')
    SRT.write("%s\n" % cell_line[i])
SRT.close()

for j in range(len(files)):
    filename = "SraRunTable"+files[j]+".txt"
    SRT = open(filename, "w")
    SRT.write("%s\n" % "sample name, treatment, cell_line")
    for i in range(len(samples)):
        if conditions[i] == 'control' or conditions[i] == files[j]:
            SRT.write(samples[i])
            SRT.write(', ')
            SRT.write(treatments[i])
            SRT.write(', ')
            SRT.write("%s\n" % cell_line[i])
    SRT.close()

print("Done")

Done
